In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [4]:
dataset_file = 'exam_2021_02_17_data.csv'
df = pd.read_csv(dataset_file, sep='\t')

print("The dataset size is {}".format(df.shape))
df.head(5)

The dataset size is (1000, 3)


,F0,F1,F2
0,d,a,a
1,c,e,c
2,j,a,c
3,f,e,j
4,c,e,d


In [11]:
labels, counts = np.unique(df, return_counts=True)
pd.DataFrame(data=[counts], columns=labels)

,a,b,c,d,e,f,g,h,i,j
0,300,298,300,300,302,300,300,300,300,300


Let's preprocess the data using an ordinal encoder

In [13]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(dtype=float)
df0 = oe.fit_transform(df)

df_ = pd.DataFrame(data=df0, columns=df.columns)
df_

,F0,F1,F2
0,3.0,0.0,0.0
1,2.0,4.0,2.0
2,9.0,0.0,2.0
3,5.0,4.0,9.0
4,2.0,4.0,3.0
...,...,...,...
995,3.0,6.0,6.0
996,1.0,8.0,5.0
997,8.0,6.0,6.0
998,3.0,7.0,1.0


Let's compute the size deviation index

In [ ]:


size_deviation_index = np.sqrt(np.unique(y, return_counts= True)[1].var())/n_clusters

Let's compare the final results

In [ ]:
from sklearn.metrics import pair_confusion_matrix
from sklearn.metrics import adjusted_rand_score

